In [2]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import minsearch
import json
import requests
import os

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
api_key = os.getenv("GROQ_API_KEY")

In [12]:
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [13]:
data = {
    "model": "llama3-70b-8192",  
    "messages": [
        {"role": "user", "content": q}
    ]
}

In [14]:
response = requests.post(url, headers=headers, json=data)

In [17]:
resp_json = response.json()
resp_json["choices"][0]["message"]["content"]

"I'm happy to help!\n\nRegarding your question, it depends on the specific course and the institution offering it. Some courses may have specific deadlines for enrollment, while others might allow late enrollment with certain restrictions.\n\nHere are a few possible scenarios:\n\n1. **Check with the course administrator or instructor**: Reach out to the person in charge of the course and ask about the possibility of late enrollment. They might have more specific information about the current status, available slots, and potential requirements for late enrollment.\n2. **Self-paced courses**: If the course is self-paced, meaning you can complete it at your own speed, you might still be able to enroll. Check the course platform or website to see if late enrollment is allowed.\n3. **Prerequisites or waitlist**: If the course has prerequisites or a waitlist, you might need to fulfill those requirements before being allowed to enroll.\n4. **Special permission**: In some cases, you might need

In [26]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

In [27]:
prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()


In [28]:
context = ""

for doc in results:
    context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"

In [31]:
prompt = prompt_template.format(
    question=q,
    context=context).strip()

In [32]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

Section: General course-related 

In [33]:
data = {
    "model": "llama3-70b-8192",  
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, headers=headers, json=data)

resp_json = response.json()
resp_json["choices"][0]["message"]["content"]

'According to the FAQ database, the answer to the question "Can I still enroll in the course now that it has already started?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.'